# Messy Stuff

Every project no matter how well organised it may appear is just a shimmering smear of oil over a sea of horribly irreproducible mess. This project is no different, apart from it doesn't look well organised to start with.

In [4]:
import pandas as pd
import numpy as np
import os
import random
import json
import tiktoken
from evaluation_funcs import load_txt_files_to_df

from openai import OpenAI
client = OpenAI()

In [41]:

#Used to modify the file name convention from the clocr-c paper which didn't really work very well. Basically just uses the article ID instead of the slug id

# def generate_filename(row):
#     """
#     Function to create a filename based on row values.
#     """
#     return f"artid_{row['id']}_periodical_{row['slug_periodical']}_issue_{row['slug_issue']}_page_number_{row.get('page_number', None)}.txt"  


# for file in os.listdir('./data/ncse_text_chunks'):

#     temp_df = pd.read_parquet('./data/ncse_text_chunks/'+file)
#     temp_df = temp_df.rename(columns={'file_name':'file_name_clocrc'})
#     temp_df['file_name'] = temp_df.apply(generate_filename, axis=1)
#     temp_df.to_parquet('./data/ncse_text_chunks2/'+file)

# Rename the Files

In the CLOCR-C paper the files used the artile 'slug' which I later discovered was not unique. As such for this project I am renaming everything to use the article ID which is a unique reference for each article. 

In [4]:
#Used to convert file names

target_dir = './data/transcripts/transcription_files'

files_in_directory =  set(os.listdir(target_dir))

file_conversion_df = []

for file in os.listdir('./data/ncse_text_chunks'):

     temp_df = pd.read_parquet('./data/ncse_text_chunks/'+file)
     temp_df = temp_df.loc[temp_df[ 'file_name_clocrc'].isin(files_in_directory), 
                           ['file_name', 'file_name_clocrc']]

     file_conversion_df.append(temp_df)

file_conversion_df = pd.concat(file_conversion_df)


In [19]:
file_conversion_df

,file_name,file_name_clocrc
5474,artid_508709_periodical_l_issue_vm2-ncseproduc...,slug_ar00602_periodical_l_issue_vm2-ncseproduc...
5475,artid_508707_periodical_l_issue_vm2-ncseproduc...,slug_ar00600_periodical_l_issue_vm2-ncseproduc...
5476,artid_508708_periodical_l_issue_vm2-ncseproduc...,slug_ar00601_periodical_l_issue_vm2-ncseproduc...
79752,artid_586629_periodical_l_issue_vm2-ncseproduc...,slug_ar00700_periodical_l_issue_vm2-ncseproduc...
79753,artid_586630_periodical_l_issue_vm2-ncseproduc...,slug_ar00701_periodical_l_issue_vm2-ncseproduc...
...,...,...
99887,artid_802839_periodical_t_issue_ttw_16051868_p...,slug_ar00508_periodical_t_issue_ttw_16051868_p...
99888,artid_802838_periodical_t_issue_ttw_16051868_p...,slug_ar00502_periodical_t_issue_ttw_16051868_p...
99889,artid_802841_periodical_t_issue_ttw_16051868_p...,slug_ar00504_periodical_t_issue_ttw_16051868_p...
99890,artid_802843_periodical_t_issue_ttw_16051868_p...,slug_ar00506_periodical_t_issue_ttw_16051868_p...


In [21]:
#rename all the files

def rename_files(row, directory):
    old_file = os.path.join(directory, row['file_name_clocrc'])
    new_file = os.path.join(directory, row['file_name'])
    try:
        os.rename(old_file, new_file)
        print(f"Renamed: {old_file} to {new_file}")
    except FileNotFoundError:
        print(f"File not found: {old_file}")
    except Exception as e:
        print(f"Error renaming {old_file} to {new_file}: {e}")

file_conversion_df.apply(rename_files, axis=1, directory=target_dir)

Renamed: ./data/transcripts/transcription_files/slug_ar00602_periodical_l_issue_vm2-ncseproduct1932_page_number_6.txt to ./data/transcripts/transcription_files/artid_508709_periodical_l_issue_vm2-ncseproduct1932_page_number_6.txt
Renamed: ./data/transcripts/transcription_files/slug_ar00600_periodical_l_issue_vm2-ncseproduct1932_page_number_6.txt to ./data/transcripts/transcription_files/artid_508707_periodical_l_issue_vm2-ncseproduct1932_page_number_6.txt
Renamed: ./data/transcripts/transcription_files/slug_ar00601_periodical_l_issue_vm2-ncseproduct1932_page_number_6.txt to ./data/transcripts/transcription_files/artid_508708_periodical_l_issue_vm2-ncseproduct1932_page_number_6.txt
Renamed: ./data/transcripts/transcription_files/slug_ar00700_periodical_l_issue_vm2-ncseproduct2102_page_number_7.txt to ./data/transcripts/transcription_files/artid_586629_periodical_l_issue_vm2-ncseproduct2102_page_number_7.txt
Renamed: ./data/transcripts/transcription_files/slug_ar00701_periodical_l_issue_

5474     None
5475     None
5476     None
79752    None
79753    None
         ... 
99887    None
99888    None
99889    None
99890    None
99892    None
Length: 91, dtype: object

In [22]:
#rename transcription files
file_conversion_df.apply(rename_files, axis=1, directory= './data/transcripts/transcription_raw_ocr')

Renamed: ./data/transcripts/transcription_raw_ocr/slug_ar00602_periodical_l_issue_vm2-ncseproduct1932_page_number_6.txt to ./data/transcripts/transcription_raw_ocr/artid_508709_periodical_l_issue_vm2-ncseproduct1932_page_number_6.txt
Renamed: ./data/transcripts/transcription_raw_ocr/slug_ar00600_periodical_l_issue_vm2-ncseproduct1932_page_number_6.txt to ./data/transcripts/transcription_raw_ocr/artid_508707_periodical_l_issue_vm2-ncseproduct1932_page_number_6.txt
Renamed: ./data/transcripts/transcription_raw_ocr/slug_ar00601_periodical_l_issue_vm2-ncseproduct1932_page_number_6.txt to ./data/transcripts/transcription_raw_ocr/artid_508708_periodical_l_issue_vm2-ncseproduct1932_page_number_6.txt
Renamed: ./data/transcripts/transcription_raw_ocr/slug_ar00700_periodical_l_issue_vm2-ncseproduct2102_page_number_7.txt to ./data/transcripts/transcription_raw_ocr/artid_586629_periodical_l_issue_vm2-ncseproduct2102_page_number_7.txt
Renamed: ./data/transcripts/transcription_raw_ocr/slug_ar00701_p

5474     None
5475     None
5476     None
79752    None
79753    None
         ... 
99887    None
99888    None
99889    None
99890    None
99892    None
Length: 91, dtype: object